 # Introducción a la ingeniería de prompts (Actualizado con Gemini)



 <!--

 Original lesson date: June 2, 2025

 This version updated to use Gemini models via OpenAI-compatible API,

 includes new sections on RAG and MCP, and is condensed for a 1-hour lesson.

 -->

 ## Configuración del Entorno



 Primero, instalaremos la biblioteca de OpenAI y configuraremos nuestra clave API para usar los modelos Gemini.

In [ ]:
# !pip install openai -q # Descomenta para instalar en Colab


 Ejecutar la siguiente celda en Colab, tras haber creado un secreto con el nombre de `GOOGLE_API_KEY` que contenga tu clave API de Gemini.

In [28]:
import os
from openai import OpenAI
import json # Para formateo de JSON y manejo de argumentos de herramientas

# Intenta obtener la clave API desde los secretos de Colab
try:
    from google.colab import userdata
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
except ImportError:
    GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY")
    if not GOOGLE_API_KEY:
        print("API Key no encontrada. Por favor, ingrésala manualmente:")
        GOOGLE_API_KEY = input()

if not GOOGLE_API_KEY:
    raise ValueError("Se requiere una Google API Key para ejecutar este notebook.")

# Configuración del cliente de OpenAI para usar Gemini
client = OpenAI(
    api_key=GOOGLE_API_KEY,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)


API Key no encontrada. Por favor, ingrésala manualmente:
Usando el modelo Gemini: gemini-2.0-flash


In [ ]:

GEMINI_MODEL = "gemini-2.0-flash" 
print(f"Usando el modelo Gemini: {GEMINI_MODEL}")


 ## ¿Qué es un Modelo de Lenguaje Grande (LLM)?



 Los LLMs son sistemas de IA entrenados para entender y generar texto similar al humano. Aprenden procesando grandes cantidades de datos textuales para predecir la siguiente palabra en un contexto dado. Su gran tamaño es clave para su potencia.



 ### Breve Historia



 * **c. 2014: RNNs (Redes Neuronales Recurrentes)** dominaban, pero tenían limitaciones con secuencias largas.

 * **Sept 2014: Attention Mechanism.** Permitió a los modelos enfocarse en partes relevantes de la entrada, superando cuellos de botella. (Referencia: *Neural Machine Translation by Jointly Learning to Align and Translate*)

 * **Jun 2017: Transformer Architecture.** Revolucionó el campo, eliminando la necesidad de RNNs. (Referencia: *Attention Is All You Need*)

 * **Jun 2018: GPT-1.** Usó solo el decodificador del Transformer para predecir la siguiente palabra. (Referencia: *Improving Language Understanding by Generative Pre-Training*)

 * **Feb 2019: GPT-2.** Mostró capacidades multitarea sin entrenamiento específico, evidenciando el poder del preentrenamiento a gran escala. (Referencia: *Language Models are Unsupervised Multitask Learners*)



 GPT-2 ya podía realizar tareas como resumen, traducción y generación de contenido, lo que planteó tanto entusiasmo como preocupaciones sobre su uso malicioso. Esto nos lleva a la importancia del **prompt crafting**.

 ## Prompt Crafting: El Arte de Hablar con las Máquinas



 Una aplicación de LLM actúa como una **capa de transformación** entre el problema del usuario y el LLM. El "prompt crafting" es el diseño cuidadoso de las entradas (prompts) para guiar al LLM.



 Exploremos algunas técnicas:

In [32]:
# Función de completado usando Gemini
def completion_gemini(user_message, system_message="You are a base language model that performs text completion. You predict the next words or tokens to complete the given text. Do not engage in conversation or provide explanations unless specifically requested. Simply complete the text naturally.", model=GEMINI_MODEL, temperature=0.7):
    try:
        messages = []
        if system_message:
            messages.append({"role": "system", "content": system_message})
        messages.append({"role": "user", "content": user_message})

        chat_completion = client.chat.completions.create(
            model=model,
            messages=messages,
            temperature=temperature,
        )
        return chat_completion.choices[0].message.content
    except Exception as e:
        return f"Error al generar la completación: {e}"


In [33]:

# Ejemplo rápido
print(completion_gemini("Explica brevemente qué es un Transformer en IA."))


Un Transformer es un modelo de red neuronal que se basa en mecanismos de autoatención, destacando por su capacidad para procesar secuencias de datos en paralelo. Esto lo hace especialmente eficiente para tareas de procesamiento del lenguaje natural como la traducción automática, la generación de texto y el análisis de sentimientos.




 ### Técnica 1: Few-Shot Prompting



 Se proporcionan al LLM algunos ejemplos de la tarea deseada para guiar su respuesta.



 ```markdown

 > How are you doing today?

 < ¿Cómo estás hoy?

 > My name is John.

 < Mi nombre es John.

 > Can I have fries with that?

 ```

 El LLM continuará el patrón.



 * **Clave:** Simplicidad y efectividad. El contexto inicial es crucial.

In [34]:
few_shot_prompt = """Q: How are you doing today?
A: ¿Cómo estás hoy?
Q: My name is John.
A: Mi nombre es John.
Q: Can I have fries with that?
A:"""


print(completion_gemini(few_shot_prompt))


¿Me da papas fritas con eso?





 ### Técnica 2: Chain of Thought Reasoning (CoT)



 Se instruye al LLM para que "piense paso a paso" o se le muestran ejemplos de razonamiento. Esto mejora la lógica en problemas complejos.



 **Ejemplo con CoT (few-shot):**

 ```markdown

 P: Jim tiene el doble de la edad de Steve. Jim tiene 12 años, ¿cuántos años tiene Steve?

 R: En forma de ecuación: 12 = 2*a donde a es la edad de Steve. Dividiendo ambos lados por 2 vemos que a = 6. Steve tiene 6 años.

 P: A un panadero le lleva una hora hacer un pastel. ¿Cuánto tiempo tardan 3 panaderos en hacer 3 pasteles?

 R:

 ```

 **Respuesta esperada (correcta):** Si cada panadero tarda 1 hora en hacer un pastel, entonces 3 panaderos tardarán 1 hora en hacer 3 pasteles.



 **Zero-shot CoT:** Simplemente añadir "Pensemos paso a paso" puede ayudar.

In [35]:
chain_of_thought_prompt = """Q: Jim tiene el doble de la edad de Steve. Jim tiene 12 años, ¿cuántos años tiene Steve?
A: En forma de ecuación: 12 = 2*a donde a es la edad de Steve. Dividiendo ambos lados por 2 vemos que a = 6. Steve tiene 6 años.
Q: A un panadero le lleva una hora hacer un pastel. ¿Cuánto tiempo tardan 3 panaderos en hacer 3 pasteles?
A:"""

print(completion_gemini(chain_of_thought_prompt))


Cada panadero tarda una hora en hacer un pastel.


In [36]:

zero_shot_cot_prompt = """Q: A un panadero le lleva una hora hacer un pastel. ¿Cuánto tiempo tardan 3 panaderos en hacer 3 pasteles?
A: Pensemos paso a paso."""

print(completion_gemini(zero_shot_cot_prompt))


Un panadero puede hacer un pastel en una hora.
Por lo tanto, 3 panaderos pueden hacer 3 pasteles en una hora.
La respuesta es 1 hora.




 ### Técnica 3: Imitación de Documentos



 Estructurar el prompt para imitar formatos conocidos (transcripciones, código, etc.) ayuda al LLM a entender el contexto.



 **Ejemplo (Soporte Técnico):**

 ```markdown

 # IT Support Assistant

 The following is a transcript between an award-winning IT support rep and a customer.



 ## Customer:

 My cable is out! And I'm going to miss the Super Bowl!



 ## Support Assistant:

 ```

 Esto le da al LLM pistas sobre el rol, la estructura y el tono.

In [37]:
document_mimicry_prompt = """# IT Support Assistant
The following is a transcript between an award-winning IT support rep and a customer.

## Customer: 
My cable is out! And I'm going to miss the Super Bowl!

## Support Assistant:"""

print(completion_gemini(document_mimicry_prompt, system_message="You are an award-winning IT support representative. Be helpful and empathetic."))


Oh no! That's terrible timing! I understand how frustrating it is to miss the Super Bowl because of a cable outage. Let's get this fixed for you ASAP. 

First, can you tell me a little more about what's happening? Are you getting any error messages on your TV screen? Is the picture just black, or is it pixelated or fuzzy? Knowing a bit more will help me troubleshoot more effectively.

In the meantime, I'll start checking for any known outages in your area. Let's get you back to the game! 



 ### Creando el Prompt: Un Proceso



 1.  **Recopilar contexto:** ¿Qué información necesita el LLM?

 2.  **Rankear contexto:** ¿Qué es lo más importante?

 3.  **Recortar contexto:** Evitar información innecesaria (los LLMs tienen ventanas de contexto limitadas).

 4.  **Ensamblar el prompt:** Combinar todo de forma clara.



 <!-- Nota para Colab: Las siguientes imágenes son referenciadas desde una carpeta local 'images'.

      Asegúrate de que esta carpeta esté en tu Google Drive y montada,

      o reemplaza 'src' con URLs directas a las imágenes si están alojadas online. -->

 <img style="width: 70%" src="images/fig-14.png" id="llm-app" alt="[Diagrama conceptual de una aplicación LLM]"/>



 *Leyenda: Diagrama conceptual de una aplicación LLM, mostrando la capa de transformación entre el usuario y el modelo.*



 ### El Chat y las "Tools" (Function Calling)



 Con la llegada de modelos conversacionales como ChatGPT, surgieron formatos como ChatML, que usan roles (`system`, `user`, `assistant`). Los **mensajes de sistema** son cruciales para guiar el comportamiento del LLM.



 <img style="width: 70%" src="images/fig-17.png" id="llm-app-chatml" alt="[Ejemplo de estructura de mensajes en ChatML]"/>

 *Leyenda: Ejemplo de estructura de mensajes en ChatML, mostrando roles de sistema, usuario y asistente.*



 El **Function Calling** (o "Tool Calling") extiende esto, permitiendo a los LLMs interactuar con APIs externas.

 ## Estructuración de Salida en Modo JSON



 Los LLMs pueden generar respuestas en JSON, útil para la integración. Se usa `response_format={"type": "json_object"}`. Para estructuras complejas, Pydantic es una buena opción.

In [40]:
# !pip install pydantic -q # Descomenta para instalar en Colab
from pydantic import BaseModel, Field
from typing import List

class Fruit(BaseModel):
    name: str = Field(description="The name of the fruit")
    color: str = Field(description="The color of the fruit")

class FruitBasket(BaseModel):
    fruits: List[Fruit] = Field(description="A list of fruits")
    description: str = Field(description="A brief description")

def chat_gemini_pydantic_json(user_prompt, system_message="You are a helpful assistant designed to output structured JSON data.", model=GEMINI_MODEL):
    try:
        messages = [
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_prompt}
        ]
        completion = client.beta.chat.completions.parse(
            model=model,
            messages=messages,
            response_format=FruitBasket
        )
        
        return completion.choices[0].message.content
    except Exception as e:
        return f"Error: {e}"

# Ejemplo Pydantic
pydantic_user_prompt = "Describe una canasta con una manzana roja y un plátano amarillo."
print(chat_gemini_pydantic_json(pydantic_user_prompt))


{
  "description": "Una canasta con una manzana roja y un plátano amarillo.",
  "fruits": [
    {
      "name": "manzana",
      "color": "roja"
    },
    {
      "name": "plátano",
      "color": "amarillo"
    }
  ]
}


 ## Tool Calling (Function Calling) con Gemini



 El "Tool Calling" permite a los LLMs interactuar con funciones externas que tú defines, ampliando sus capacidades más allá de la generación de texto. Es como darle al LLM herramientas para realizar tareas específicas.



 **Flujo Detallado:**



 1.  **Definición de la Herramienta (Tu Código Python):**

     * Creas una función Python normal que realiza una tarea específica (ej., obtener el clima, buscar en una base de datos, calcular algo).

     * Esta función debe devolver idealmente una cadena (a menudo JSON) que el LLM pueda entender.



 2.  **Especificación de la Herramienta para el LLM (JSON Schema):**

     * Le describes esta función al LLM usando un formato específico (JSON Schema).

     * Esta descripción incluye:

         * `name`: Nombre de la función (debe coincidir con tu función Python).

         * `description`: Qué hace la función (para que el LLM sepa cuándo usarla).

         * `parameters`: Qué argumentos necesita la función, sus tipos y descripciones.



 3.  **Llamada Inicial al LLM:**

     * Envías el prompt del usuario al LLM.

     * Junto con el prompt, envías la lista de `tools` (las especificaciones de tus herramientas).

     * Usas `tool_choice="auto"` para que el LLM decida si necesita usar alguna herramienta para responder al prompt del usuario.



 4.  **Respuesta del LLM (Decisión de Usar Herramienta):**

     * Si el LLM determina que necesita una herramienta, en lugar de una respuesta textual directa, su mensaje contendrá `tool_calls`.

     * Cada `tool_call` indica:

         * `id`: Un identificador único para esta llamada.

         * `function.name`: El nombre de la función que quiere ejecutar.

         * `function.arguments`: Una cadena JSON con los argumentos que el LLM cree que tu función necesita.



 5.  **Ejecución de la Herramienta (Tu Código):**

     * Tu código recibe estos `tool_calls`.

     * Para cada `tool_call`:

         * Identificas qué función Python ejecutar basándote en `function.name`.

         * Parseas `function.arguments` (de JSON a un diccionario Python).

         * Llamas a tu función Python con esos argumentos.



 6.  **Envío de Resultados al LLM:**

     * Añades un nuevo mensaje al historial de la conversación. Este mensaje tiene:

         * `role`: "tool"

         * `tool_call_id`: El mismo ID de la `tool_call` original.

         * `name`: El nombre de la función que se ejecutó.

         * `content`: El resultado que devolvió tu función Python.



 7.  **Llamada Final al LLM:**

     * Vuelves a llamar al LLM con el historial de conversación actualizado (que ahora incluye la solicitud de herramienta y su resultado).

     * El LLM usará el resultado de la herramienta para formular una respuesta final y natural al prompt original del usuario.



 <img style="width: 70%" src="images/fig-18.png" id="llm-app-tools" alt="[Diagrama de flujo de Function Calling]"/>

 *Leyenda: Flujo simplificado de Function Calling, mostrando la interacción entre el usuario, el LLM y las herramientas externas.*

In [43]:
from pydantic import BaseModel, Field
from typing import Optional, Literal
import json

# Modelos Pydantic para definir las herramientas
class WeatherRequest(BaseModel):
    location: str = Field(description="La ciudad, ej. San Francisco, CA")
    unit: Literal["celsius", "fahrenheit"] = Field(default="celsius", description="Unidad de temperatura")

class WeatherResponse(BaseModel):
    location: str
    temperature: str
    unit: str
    condition: str
    error: Optional[str] = None

# Función de herramienta simplificada
def get_current_weather(request: WeatherRequest) -> WeatherResponse:
    """Obtiene el clima actual para una ubicación dada (simulado)."""
    weather_data = {
        "tokyo": {"temperature": "15", "condition": "Soleado"},
        "london": {"temperature": "10", "condition": "Nublado"},
        "paris": {"temperature": "12", "condition": "Lluvioso"}
    }
    
    city = request.location.lower().split(",")[0]
    
    if city in weather_data:
        return WeatherResponse(
            location=request.location,
            temperature=weather_data[city]["temperature"],
            unit=request.unit,
            condition=weather_data[city]["condition"]
        )
    
    return WeatherResponse(
        location=request.location,
        temperature="",
        unit=request.unit,
        condition="",
        error="Clima no encontrado"
    )

# Generación automática de tool specs desde Pydantic
def generate_tool_spec(model_class: BaseModel, function_name: str, description: str):
    """Genera automáticamente la especificación de herramienta desde un modelo Pydantic."""
    return {
        "type": "function",
        "function": {
            "name": function_name,
            "description": description,
            "parameters": model_class.model_json_schema()
        }
    }

# Tool specs generadas automáticamente
tools_specs = [
    generate_tool_spec(
        WeatherRequest, 
        "get_current_weather", 
        "Obtiene el clima actual en una ubicación específica."
    )
]

# Registro de herramientas disponibles
AVAILABLE_TOOLS = {
    "get_current_weather": {
        "function": get_current_weather,
        "input_model": WeatherRequest
    }
}

def run_conversation_with_tools(user_prompt: str, verbose: bool = False) -> str:
    """Ejecuta una conversación con herramientas usando Pydantic para validación."""
    messages = [{"role": "user", "content": user_prompt}]
    if verbose: print(f"User: {user_prompt}\n")

    try:
        # Primera llamada al LLM
        response = client.chat.completions.create(
            model=GEMINI_MODEL,
            messages=messages,
            tools=tools_specs,
            tool_choice="auto"
        )
        
        response_message = response.choices[0].message
        if verbose: print(f"LLM decidió: {'usar herramientas' if response_message.tool_calls else 'responder directamente'}\n")

        # Si no hay tool calls, devolver respuesta directa
        if not response_message.tool_calls:
            return response_message.content

        # Procesar tool calls
        messages.append(response_message)
        
        for tool_call in response_message.tool_calls:
            function_name = tool_call.function.name
            
            if function_name not in AVAILABLE_TOOLS:
                if verbose: print(f"Herramienta no encontrada: {function_name}\n")
                continue
                
            tool_info = AVAILABLE_TOOLS[function_name]
            
            # Validar argumentos con Pydantic
            try:
                function_args = json.loads(tool_call.function.arguments)
                validated_input = tool_info["input_model"](**function_args)
                if verbose: print(f"Ejecutando: {function_name} con {validated_input.model_dump()}\n")
                
                # Ejecutar función
                result = tool_info["function"](validated_input)
                tool_response = result.model_dump_json()
                if verbose: print(f"Resultado: {tool_response}\n")
                
            except Exception as e:
                tool_response = json.dumps({"error": f"Error validando argumentos: {str(e)}"})
                if verbose: print(f"Error: {tool_response}\n")
            
            # Añadir respuesta de herramienta al historial
            messages.append({
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": tool_response
            })
        
        # Segunda llamada al LLM con resultados de herramientas
        if verbose: print("Generando respuesta final...\n")
        final_response = client.chat.completions.create(
            model=GEMINI_MODEL, 
            messages=messages
        )
        
        return final_response.choices[0].message.content
        
    except Exception as e:
        return f"Error en Tool Calling: {e}"

# Ejemplo de uso
user_weather_prompt = "What's the weather like in London today?"
final_answer = run_conversation_with_tools(user_weather_prompt, verbose=True)
print(f"\nRespuesta final: {final_answer}")

User: What's the weather like in London today?

LLM decidió: usar herramientas

Ejecutando: get_current_weather con {'location': 'London', 'unit': 'celsius'}

Resultado: {"location":"London","temperature":"10","unit":"celsius","condition":"Nublado","error":null}

Generando respuesta final...


Respuesta final: The weather in London today is Nublado with a temperature of 10 degrees Celsius.


```mermaid
sequenceDiagram
    participant U as 👤 Usuario
    participant M as 📝 Main Function
    participant L as 🤖 LLM (Gemini)
    participant P as 🛡️ Pydantic
    participant T as ⚙️ Tool Function
    
    U->>M: "What's the weather in London?"
    M->>L: Primera llamada con tools_specs
    
    Note over L: Analiza si necesita herramientas
    L->>M: response_message con tool_calls
    
    Note over M: Para cada tool_call
    M->>P: Validar argumentos con WeatherRequest
    P->>M: validated_input: WeatherRequest(location="London", unit="celsius")
    
    M->>T: get_current_weather(validated_input)
    T->>M: WeatherResponse(location="London", temperature="10", ...)
    
    M->>M: Convertir a JSON con model_dump_json()
    M->>L: Segunda llamada con tool results
    
    Note over L: Procesa resultados y genera respuesta natural
    L->>M: "The weather in London is..."
    M->>U: Respuesta final
    
    rect rgb(240, 248, 255)
        Note over P: 🛡️ Pydantic garantiza:<br/>- Validación de tipos<br/>- Estructura correcta<br/>- Manejo de errores
    end
    
    rect rgb(248, 255, 240)
        Note over T: ⚙️ Tool Function:<br/>- Recibe objeto tipado<br/>- Retorna objeto tipado<br/>- Lógica de negocio limpia
    end
```

## Retrieval Augmented Generation (RAG)



 **RAG** mejora los LLMs al darles acceso a información externa y actualizada.



 **Flujo Básico:**

 1.  **Consulta del Usuario.**

 2.  **Recuperación (Retrieval):** Buscar información relevante en una base de datos (corpus).

 3.  **Aumentación (Augmentation):** Combinar la información recuperada con la consulta original.

 4.  **Generación (Generation):** El LLM usa el prompt enriquecido para responder.



 **Beneficios:** Reduce alucinaciones, usa datos actuales, permite citar fuentes.



 ### Ejemplo Básico de RAG

In [42]:
# Configuración del modelo de embeddings
EMBEDDINGS_MODEL = "gemini-embedding-exp-03-07"

# Ejemplo básico de embeddings
prompt = """
    The quick brown fox jumps over the lazy dog.
"""

response = client.embeddings.create(
    model=EMBEDDINGS_MODEL,
    input=prompt,
)

print(len(response.data[0].embedding))
print(response.data[0].embedding[:4], '...')

# Corpus de documentos mejorado para RAG
document_corpus = {
    "doc1": "El Sol es una estrella en el centro de nuestro sistema solar.",
    "doc2": "La Luna es el satélite natural de la Tierra.",
    "doc3": "Marte, el 'planeta rojo', tiene dos lunas: Fobos y Deimos."
}

def get_embedding(text, model=EMBEDDINGS_MODEL):
    """Obtiene el embedding de un texto usando Gemini."""
    try:
        response = client.embeddings.create(
            model=model,
            input=text,
        )
        return response.data[0].embedding
    except Exception as e:
        print(f"Error al obtener embedding: {e}")
        return None

def cosine_similarity(vec1, vec2):
    """Calcula la similitud coseno entre dos vectores."""
    import math
    dot_product = sum(a * b for a, b in zip(vec1, vec2))
    magnitude1 = math.sqrt(sum(a * a for a in vec1))
    magnitude2 = math.sqrt(sum(a * a for a in vec2))
    if magnitude1 == 0 or magnitude2 == 0:
        return 0
    return dot_product / (magnitude1 * magnitude2)

def retrieve_relevant_documents(query, corpus, top_k=1):
    """Recupera documentos relevantes usando embeddings para mejor precisión."""
    query_embedding = get_embedding(query)
    if query_embedding is None:
        return []
    
    # Calcular embeddings para todos los documentos
    doc_embeddings = {}
    for doc_id, text in corpus.items():
        embedding = get_embedding(text)
        if embedding:
            doc_embeddings[doc_id] = embedding
    
    # Calcular similitudes
    doc_scores = {}
    for doc_id, doc_embedding in doc_embeddings.items():
        similarity = cosine_similarity(query_embedding, doc_embedding)
        doc_scores[doc_id] = similarity
    
    # Ordenar por similitud y devolver los top_k
    sorted_docs = sorted(doc_scores.items(), key=lambda item: item[1], reverse=True)
    return [corpus[doc_id] for doc_id, score in sorted_docs[:top_k] if score > 0]

def rag_generate_answer(user_query, corpus, model=GEMINI_MODEL):
    retrieved_texts = retrieve_relevant_documents(user_query, corpus)
    context_for_llm = "\n\n".join(retrieved_texts) if retrieved_texts else "No se encontró información específica."
    
    system_message_rag = "Responde basándote en el contexto. Si no es suficiente, indícalo."
    prompt_with_context = f"Contexto:\n{context_for_llm}\n\nPregunta: {user_query}\nRespuesta:"
    return completion_gemini(prompt_with_context, system_message=system_message_rag, model=model)

query_rag = "¿Cuántas lunas tiene Marte?"
print(f"\nPregunta RAG: {query_rag}\nRespuesta RAG: {rag_generate_answer(query_rag, document_corpus)}")


3072
[-0.009576111100614071, 0.024940399453043938, 0.00013856856094207615, -0.04367725923657417] ...

Pregunta RAG: ¿Cuántas lunas tiene Marte?
Respuesta RAG: Marte tiene dos lunas.



```mermaid
flowchart TD
    A["👤 Usuario hace pregunta:<br/>¿Cuántas lunas tiene Marte?"] --> B["🔍 Obtener embedding<br/>de la consulta"]
    
    C["📚 Corpus de documentos:<br/>doc1: El Sol es una estrella...<br/>doc2: La Luna es el satélite...<br/>doc3: Marte tiene dos lunas..."] --> D["🔢 Obtener embeddings<br/>de todos los documentos"]
    
    B --> E["📊 Calcular similitud coseno<br/>entre consulta y cada documento"]
    D --> E
    
    E --> F["📋 Ordenar documentos<br/>por similitud (descendente)"]
    
    F --> G["🎯 Seleccionar top_k<br/>documentos más relevantes"]
    
    G --> H["📝 Construir contexto<br/>combinando textos recuperados"]
    
    H --> I["🤖 Enviar al LLM con prompt:<br/>Contexto: [documentos recuperados]<br/>Pregunta: [consulta original]"]
    
    I --> J["✅ Respuesta generada:<br/>Marte tiene dos lunas"]
    
    subgraph "🔧 Funciones del proceso"
        B1["get_embedding()"]
        B2["cosine_similarity()"]
        B3["retrieve_relevant_documents()"]
        B4["rag_generate_answer()"]
    end
    
    subgraph "📈 Datos vectoriales"
        V1["Vector consulta:<br/>[0.1, -0.3, 0.7, ...]"]
        V2["Vector doc1:<br/>[0.2, -0.1, 0.4, ...]"]
        V3["Vector doc2:<br/>[0.5, 0.2, -0.1, ...]"]
        V4["Vector doc3:<br/>[0.1, -0.2, 0.8, ...]"]
    end
    
    B -.-> B1
    E -.-> B2
    G -.-> B3
    I -.-> B4
    
    B --> V1
    D --> V2
    D --> V3
    D --> V4
    
    style A fill:#e1f5fe
    style J fill:#e8f5e8
    style G fill:#fff3e0
    style I fill:#f3e5f5
    ```

## Model Context Protocol (MCP) 

### ¿Qué es MCP?

El **Model Context Protocol (MCP)** es un estándar abierto desarrollado por Anthropic que permite conectar modelos de IA (como Claude, ChatGPT, etc.) con sistemas externos y fuentes de datos de manera estandarizada. Es como un "protocolo universal" que resuelve el problema N×M: en lugar de crear integraciones personalizadas para cada combinación de modelo de IA y herramienta externa, MCP proporciona una interfaz común.

### ¿Cómo se usa?

MCP utiliza una arquitectura **cliente-servidor**:

- **MCP Client**: Integrado en la aplicación de IA (como Claude Desktop)
- **MCP Server**: Expone datos y herramientas específicas (GitHub, Google Drive, bases de datos, etc.)

**Flujo típico:**
1. El usuario hace una pregunta al modelo de IA
2. El cliente MCP identifica qué servidores pueden ayudar
3. Se obtiene información de los servidores MCP relevantes
4. El modelo genera una respuesta usando ese contexto adicional

### ¿Cómo implementarlo?

#### Ejemplo básico de servidor MCP:

```python
# Estructura simplificada de un servidor MCP
class MCPServer:
    def list_tools(self):
        """Retorna herramientas disponibles"""
        return [{
            "name": "get_user_data",
            "description": "Obtiene datos del usuario",
            "parameters": {"user_id": "string"}
        }]
    
    def call_tool(self, name, arguments):
        """Ejecuta la herramienta solicitada"""
        if name == "get_user_data":
            return fetch_user_from_database(arguments["user_id"])
```

#### Integración con Claude Desktop:
```json
{
  "mcpServers": {
    "my-server": {
      "command": "python",
      "args": ["my_mcp_server.py"]
    }
  }
}
```

### Beneficios Clave

- **Estandarización**: Un protocolo para todas las integraciones
- **Interoperabilidad**: Cualquier cliente MCP puede usar cualquier servidor MCP
- **Escalabilidad**: Fácil agregar nuevas fuentes de datos sin reescribir código

### Referencias

1. **Anthropic Official Announcement**: [Introducing the Model Context Protocol](https://www.anthropic.com/news/model-context-protocol) - Announcement oficial de Anthropic del 25 de noviembre de 2024

2. **Technical Deep Dive**: [What Is the Model Context Protocol (MCP) and How It Works](https://www.descope.com/learn/post/mcp) - Guía técnica detallada publicada en abril de 2025

MCP está diseñado para ser el "Language Server Protocol" del mundo de la IA, simplificando dramáticamente cómo los modelos de lenguaje acceden a información y herramientas externas.



 ## Conclusión



 El prompt crafting es un arte y una ciencia. Entender y aplicar técnicas como few-shot, CoT, imitación de documentos, RAG, MCP, y tool calling nos permite aprovechar el potencial de los LLMs de manera responsable.



 **Recurso Adicional:** [https://www.promptingguide.ai/](https://www.promptingguide.ai/)



**Uso de Langchain con Gemini**: [gemini-langchain-cheatsheet](https://www.philschmid.de/gemini-langchain-cheatsheet)

 ---

 Fin de la lección.